### Paso 1: .doc a .csv

In [1]:
import os
import re
import pandas as pd
from docx import Document

In [2]:
# Ruta a carpeta local con archivos .docx y donde se guardara el archivo .csv
carpeta = "Informes/"
archivo_csv = "documentos_medicos.csv"
datos = []
nuevos_datos = []

In [3]:
def extraer_texto_docx(archivo):
    doc = Document(archivo)
    texto = '\n'.join([p.text for p in doc.paragraphs if p.text.strip() != ""])
    return texto

In [4]:
def extraer_campos(texto):
    paciente = re.search(r"Paciente:\s*(.*)", texto)
    edad = re.search(r"Edad:\s*(\d+)", texto)
    fecha = re.search(r"Fecha de Ingreso:\s*(.*)", texto)
    dx = re.search(r"Diagnóstico Presuntivo:\s*(.*)", texto)
    meds = re.search(r"Medicamentos Recetados:(.*?)(Instrucciones|Firma|$)", texto, re.DOTALL)

    return {
        "paciente": paciente.group(1).strip() if paciente else "",
        "edad": int(edad.group(1)) if edad else "",
        "fecha_ingreso": fecha.group(1).strip() if fecha else "",
        "diagnostico": dx.group(1).strip() if dx else "",
        "medicamentos": meds.group(1).strip().replace("\n", ", ") if meds else ""
    }

In [5]:
# Cargar CSV existente si ya existe
if os.path.exists(archivo_csv):
    df_existente = pd.read_csv(archivo_csv, encoding='utf-8')
else:
    df_existente = pd.DataFrame()

In [6]:
for archivo in os.listdir(carpeta):
    if archivo.endswith(".docx"):
        ruta = os.path.join(carpeta, archivo)
        texto = extraer_texto_docx(ruta)
        campos = extraer_campos(texto)
        campos["archivo"] = archivo
        datos.append(campos)

        # Verificar si el paciente existe
        if not df_existente.empty and campos["paciente"] in df_existente["paciente"].values:
            print(f"Paciente ya registrado: {campos['paciente']} — ignorado.")
        else:
            nuevos_datos.append(campos)

# Agregar y guardar
if nuevos_datos:
    df_nuevos = pd.DataFrame(nuevos_datos)
    df_final = pd.concat([df_existente, df_nuevos], ignore_index=True)
    df_final.to_csv(archivo_csv, index=False, encoding='utf-8')
    print("Datos nuevos agregados al CSV.")
else:
    print("No se encontraron pacientes nuevos.")

Paciente ya registrado: Juan Pérez — ignorado.
No se encontraron pacientes nuevos.


In [7]:
# Guardar en CSV
df = pd.DataFrame(datos)
df.to_csv("documentos_medicos.csv", index=False, encoding='utf-8')

# Mostrar el contenido del CSV
print(df.head())

     paciente  edad        fecha_ingreso                 diagnostico  \
0  Juan Pérez    45  15 de abril de 2025  Posible apendicitis aguda.   

                                        medicamentos    archivo  
0  Paracetamol 500mg: 1 tableta cada 8 horas para...  tets.docx  


In [8]:
# Leer el archivo CSV
df = pd.read_csv("documentos_medicos.csv", encoding='utf-8')

# Mostrar las primeras filas
print(df.head())

     paciente  edad        fecha_ingreso                 diagnostico  \
0  Juan Pérez    45  15 de abril de 2025  Posible apendicitis aguda.   

                                        medicamentos    archivo  
0  Paracetamol 500mg: 1 tableta cada 8 horas para...  tets.docx  


### Paso 2: Entrenar el modelo